In [3]:
#!pip install nest_asyncio
#!pip install --upgrade telethon

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from telethon import TelegramClient
from datetime import datetime
import nest_asyncio
import pytz
import re

# Enable nested event loops for Jupyter
nest_asyncio.apply()
load_dotenv()

# API credentials
api_id = os.environ.get("TELEGRAM_API_ID")
api_hash = os.environ.get("TELEGRAM_API_HASH")
group_id = -1002041357608
topic_id = 19940
finnish_tz = pytz.timezone('Europe/Helsinki')

# Convert Finnish time to UTC-aware datetime
def convert_to_utc(dt):
    return finnish_tz.localize(dt).astimezone(pytz.utc)

async def get_topic_messages(starttime=None, endtime=None, limit=1000):
    async with TelegramClient('session_name', api_id, api_hash) as client:
        await client.start()
        
        # Convert start/end times to UTC-aware datetimes
        starttime_utc = convert_to_utc(starttime) if starttime else None
        endtime_utc = convert_to_utc(endtime) if endtime else None

        messages = []
        total_fetched = 0

        # We will iterate forward in time starting from starttime_utc
        async for message in client.iter_messages(
            entity=group_id,
            offset_date=starttime_utc,  # Start from around starttime
            reverse=True                # Move forward in time
        ):
            # If message is after endtime, stop
            if endtime_utc and message.date > endtime_utc:
                break
            # If message is before starttime, skip it
            if starttime_utc and message.date < starttime_utc:
                continue

            # Check if this message belongs to the desired topic
            # Note: forum_topic_id may differ based on Telethon version;
            # if not available, print message attributes to find a suitable field.
            if message.reply_to and message.reply_to.reply_to_msg_id == topic_id:
                sender = await message.get_sender()
                sender_name = sender.first_name if sender else "Unknown"
                messages.append({
                    'date': message.date.astimezone(finnish_tz),
                    'sender': sender_name,
                    'text': message.text,
                })

                total_fetched += 1
                if total_fetched >= limit:
                    break

        return messages

        

async def get_messages():
    async with TelegramClient('session_name', api_id, api_hash) as client:
        # Get messages
        messages = await client.get_messages(group_username, limit=10)
        
        # Print messages
        for message in messages:
            print(f'[{message.date}] {message.sender.first_name}: {message.text}')


async def list_dialogs():
    async with TelegramClient('session_name', api_id, api_hash) as client:
        dialogs = await client.get_dialogs()
        for dialog in dialogs:
            print(f"Name: {dialog.name}, ID: {dialog.id}")


# List the Recent dialogs
#await list_dialogs()

# Get the message for the whole group (dialog)
#await get_messages()

# Get the message for a specific topic within the date limit
starttime = datetime(2025, 1, 30, 0, 0, 0) 
endtime = datetime(2025, 1, 31, 0, 0, 0)
messages = await get_topic_messages(starttime=starttime, endtime=endtime)


In [2]:
df = pd.DataFrame(messages)
df

,date,sender,text
0,2025-01-30 00:32:02+02:00,繪圖員 - 娜美,"Buy - Price=103940.67, TP:104337.64, SL:103543..."
1,2025-01-30 00:36:50+02:00,繪圖員 - 娜美,"Buy - Price=103959.91, TP:104351.02, SL:103568..."
2,2025-01-30 01:06:51+02:00,繪圖員 - 娜美,"Buy - Price=103914.3, TP:104307.37000000001, S..."
3,2025-01-30 01:11:49+02:00,繪圖員 - 娜美,"Buy - Price=103979.75, TP:104373.7, SL:103585.8"
4,2025-01-30 01:16:45+02:00,繪圖員 - 娜美,"Buy - Price=104086.94, TP:104482.35, SL:103691.53"
5,2025-01-30 01:22:04+02:00,繪圖員 - 娜美,"Buy - Price=104108.6, TP:104501.40000000001, S..."
6,2025-01-30 01:26:47+02:00,繪圖員 - 娜美,"Buy - Price=104064.06, TP:104458.36, SL:103669..."
7,2025-01-30 03:25:40+02:00,繪圖員 - 娜美,"Buy - Price=154.615, TP:154.75, SL:154.4800000..."
8,2025-01-30 03:30:38+02:00,繪圖員 - 娜美,"Buy - Price=154.622, TP:154.757, SL:154.487000..."
9,2025-01-30 03:36:30+02:00,繪圖員 - 娜美,"Buy - Price=0.56609, TP:0.56655, SL:0.56563\nS..."


In [3]:
# Function to extract desired fields using regex
def extract_fields(text):
    # Initialize default values
    type_ = None
    symbol = None
    price = None
    lot = None
    tp = None
    sl = None
    
    # Extract Type: 'Buy' or 'Sell'
    type_match = re.match(r'^(Buy|Sell)', text, re.IGNORECASE)
    if type_match:
        type_ = type_match.group(1).capitalize()
    
    # Extract Symbol: text after 'Sending order ' and before ' Lot:'
    symbol_match = re.search(r'Sending order\s+(\w+)\s+Lot:', text, re.IGNORECASE)
    if symbol_match:
        symbol = symbol_match.group(1)
    
    # Extract Price from the first part (after 'Price=')
    price_match = re.search(r'Price=([\d\.]+)', text)
    if price_match:
        price = float(price_match.group(1))
    
    # Extract Lot: after 'Lot:'
    lot_match = re.search(r'Lot:\s*([\d\.]+)', text, re.IGNORECASE)
    if lot_match:
        lot = float(lot_match.group(1))
    
    # Extract TP: after 'TP:'
    tp_match = re.search(r'TP:\s*([\d\.]+)', text, re.IGNORECASE)
    if tp_match:
        tp = float(tp_match.group(1))
    
    # Extract SL: after 'SL:'
    sl_match = re.search(r'SL:\s*([\d\.]+)', text, re.IGNORECASE)
    if sl_match:
        sl = float(sl_match.group(1))
    
    return pd.Series({
        'Type': type_,
        'Symbol': symbol,
        'Price': price,
        'Lot': lot,
        'TP': tp,
        'SL': sl
    })

# Apply the extraction function to each row in the 'text' column
filtered_df = df[df['text'].str.contains("Sending order")].copy()
filtered_df[['Type', 'Symbol', 'Price', 'Lot', 'TP', 'SL']] = filtered_df['text'].apply(extract_fields)
filtered_df


,date,sender,text,Type,Symbol,Price,Lot,TP,SL
2,2025-01-30 01:06:51+02:00,繪圖員 - 娜美,"Buy - Price=103914.3, TP:104307.37000000001, S...",Buy,BTCUSD,103914.30000,0.29,104307.37000,103521.23000
7,2025-01-30 03:25:40+02:00,繪圖員 - 娜美,"Buy - Price=154.615, TP:154.75, SL:154.4800000...",Buy,USDJPY,154.61500,1.37,154.75000,154.48000
9,2025-01-30 03:36:30+02:00,繪圖員 - 娜美,"Buy - Price=0.56609, TP:0.56655, SL:0.56563\nS...",Buy,NZDUSD,0.56609,2.64,0.56655,0.56563
12,2025-01-30 10:21:14+02:00,繪圖員 - 娜美,"Buy - Price=0.83683, TP:0.83716, SL:0.83649999...",Buy,EURGBP,0.83683,2.74,0.83716,0.83650
13,2025-01-30 10:21:24+02:00,繪圖員 - 娜美,"Sell - Price=1.24535, TP:1.24443, SL:1.24627\n...",Sell,GBPUSD,1.24535,1.21,1.24443,1.24627


In [4]:
filtered_df['datelocal'] = (
    filtered_df['date']
    .dt.tz_convert(finnish_tz)  # Convert to Europe/Helsinki timezone
    .dt.tz_localize(None)        # Remove timezone information
)
filtered_df[['datelocal','Symbol','Type','Price']].to_excel('maptrade/maptrade_signal.xlsx')